In [203]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [204]:
df=pd.read_csv('/Users/pranav/Desktop/Coding/Data-Science/DATA/Churn_Modelling.csv')

df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)
df['Geography'].value_counts()

label_encoder=LabelEncoder()
df['Gender']=label_encoder.fit_transform(df['Gender'])

ohe=OneHotEncoder(sparse_output=False)
new_df=pd.DataFrame(ohe.fit_transform(df[['Geography']]),columns=ohe.get_feature_names_out())

df=pd.concat([df,new_df],axis=1)
df=df.drop(columns='Geography')

In [205]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [206]:
X=df.drop(columns='Exited')
y=df['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# ANN

In [225]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

(X_train.shape[1],)

(12,)

In [208]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1
    Dense(32,activation='relu'),  #HL2
    Dense(1,activation='sigmoid')    # Output layer
])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [231]:
loss=keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=False,
    alpha=0.25,
    gamma=2.0,
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="sum_over_batch_size",
    name="binary_focal_crossentropy",
    dtype=None,
)

model.compile(
    loss=loss,
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

batch_size = 128
epochs = 20

early_stopper=keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

tensorboard=keras.callbacks.TensorBoard(
    log_dir="logs",
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq="epoch",
    profile_batch=0,
    embeddings_freq=0,
    embeddings_metadata=None,
)

model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=[early_stopper,tensorboard]
)
model.evaluate(X_test,y_test)

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.7899 - loss: 0.1167 - val_acc: 0.7983 - val_loss: 0.1061
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.7930 - loss: 0.0862 - val_acc: 0.7983 - val_loss: 0.1025
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.7908 - loss: 0.0797 - val_acc: 0.7983 - val_loss: 0.0978
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.7923 - loss: 0.0792 - val_acc: 0.7983 - val_loss: 0.0985
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - acc: 0.8026 - loss: 0.0991


[0.10159879922866821, 0.8034999966621399]

In [242]:
y_pred = (model.predict(X_test)>0.5).astype("int32")


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step


In [243]:
from sklearn.metrics import confusion_matrix,accuracy_score,mean_absolute_error

mean_absolute_error(y_test,y_pred)
confusion_matrix(y_test,y_pred)
accuracy_score(y_pred,y_test)

0.845